# Preprocess cholera outbreaks

This notebook executes a Python script to download weekly cholera outbreaks data in PDF format from the [Integrated Disease Surveillance Programme of India](https://idsp.nic.in/) (IDSP). It then cleans and prepares the data and saves it in shapefile format for further processing.

In [1]:
# import packages
import os
import numpy as np
import pandas as pd
import PyPDF2
import re
import geopandas as gpd
import requests
import zipfile
import io

## Download cholera outbreaks data

Run Python script to download cholera outbreaks data. Before downloading a file the script checks whether that file has already been downloaded or not.

In [2]:
!python download_cholera_outbreaks_data.py

Download cholera outbreaks data...
Processing 2010 with 53 files...
Processing 2011 with 52 files...
Processing 2012 with 52 files...
Processing 2013 with 52 files...
Processing 2014 with 52 files...
Processing 2015 with 53 files...
Processing 2016 with 52 files...
Processing 2017 with 52 files...
Processing 2018 with 52 files...
Download cholera outbreaks data complete.


## Load PDFs and extract relevant parts

In [3]:
# make list of years that are covered by the data
years = list(np.arange(2010, 2019))

In [4]:
# set path for cholera outbreaks data
path = '../data/cholera_outbreaks'

Next, all PDFs are loaded in a for loop and the relevant information is extracted.

In [5]:
%%time

outbreaks_raw = []

for year in years:
    print('Processing year: {}...'.format(year))
    (_, _, file_names) = next(os.walk(os.path.join(path, str(year)))) # get all file names in directory
    
    for file in file_names:
        print('Processing file: {}...'.format(file))
        document = PyPDF2.PdfFileReader(os.path.join(path, str(year), file)) # load pdf
        pages = document.getNumPages() # get number of pages
        
        for page in range(pages):
            text = document.getPage(page).extractText() # extract raw text from pdf
            text = text.replace('\n', '') # remove line breaks
            text = text.split(' ') # split on space
            text = [i for i in text if i != ''] # remove empty items
            text = [i.lower() for i in text] # lower case all items
    
            indices = [i for i, s in enumerate(text) if 'cholera' in s] # get indices where cholera is mentioned
        
            # if cholera is mentioned on page proceed with extraction of information        
            if len(indices) > 0:
                for index in indices:
                    cholera_index = []
                    cholera_index.append(index) # get index where cholera is mentioned
                    index_list = np.arange(index-3,index).tolist() + cholera_index + np.arange(index+1,index+5).tolist() # construct list to extract neighboring information
                    index_list = [i for i in index_list if i < len(text)] # get items that are up to 3 indices before and up to 5 indices after where the index where cholera is mentioned
                    outbreak_info = [text[i] for i in index_list]
                    # append extracted information to temporary list
                    while len(outbreak_info) < 9:
                        outbreak_info.append(None)
                    # append file name to temporary list
                    outbreak_info.append(str(year)+'/'+file)
                    # append extracted information to final list
                    outbreaks_raw.append(outbreak_info)
            # continue otherwise
            else:
                pass

Processing year: 2010...
Processing file: 31st_2010.pdf...
Processing file: 3rd_2010.pdf...
Processing file: 5th_2010.pdf...
Processing file: 42nd_2010.pdf...
Processing file: 39th_2010.pdf...
Processing file: 41st_2010.pdf...
Processing file: 27th_2010.pdf...
Processing file: 53rd_2010.pdf...
Processing file: 9th_2010.pdf...
Processing file: 33rd_2010.pdf...
Processing file: 25th_2010.pdf...
Processing file: 28th_2010.pdf...
Processing file: 14th_2010.pdf...
Processing file: 47th_2010.pdf...
Processing file: 21st_2010.pdf...
Processing file: 8th_2010.pdf...
Processing file: 17th_2010.pdf...
Processing file: 40th_2010.pdf...
Processing file: 1st_2010.pdf...
Processing file: 52nd_2010.pdf...
Processing file: 6th_2010.pdf...
Processing file: 35th_2010.pdf...
Processing file: 23rd_2010.pdf...
Processing file: 18th_2010.pdf...
Processing file: 44th_2010.pdf...
Processing file: 36th_2010.pdf...
Processing file: 11th_2010.pdf...
Processing file: 24th_2010.pdf...
Processing file: 2nd_2010.pdf

Processing file: 4th_2011.pdf...
Processing file: 41st_2011.pdf...
Processing file: 44th_2011.pdf...
Processing file: 39th_2011.pdf...
Processing file: 11th_2011.pdf...
Processing file: 49th_2011.pdf...
Processing file: 52nd_2011.pdf...
Processing file: 17th_2011.pdf...
Processing file: 15th_2011.pdf...
Processing file: 25th_2011.pdf...
Processing file: 51st_2011.pdf...
Processing file: 9th_2011.pdf...
Processing file: 29th_2011.pdf...
Processing file: 16th_2011.pdf...
Processing file: 40th_2011.pdf...
Processing file: 2nd_2011.pdf...
Processing file: 50th_2011.pdf...
Processing file: 13th_2011.pdf...
Processing file: 42nd_2011.pdf...
Processing file: 34th_2011.pdf...
Processing file: 12th_2011.pdf...
Processing file: 38th_2011.pdf...
Processing file: 36th_2011.pdf...
Processing file: 14th_2011.pdf...
Processing file: 30th_2011.pdf...
Processing file: 24th_2011.pdf...
Processing file: 45th_2011.pdf...
Processing file: 27th_2011.pdf...
Processing file: 46th_2011.pdf...
Processing file: 

Processing file: 20th_2017.pdf...
Processing file: 4th_2017.pdf...
Processing file: 15th_2017.pdf...
Processing file: 19th_2017.pdf...
Processing file: 41th_2017.pdf...
Processing file: 31st_2017.pdf...
Processing file: 14th_2017.pdf...
Processing file: 46th_2017.pdf...
Processing file: 47th_2017.pdf...
Processing file: 25th_2017.pdf...
Processing file: 39th_2017.pdf...
Processing file: 13th_2017.pdf...
Processing file: 21st_2017.pdf...
Processing file: 52nd_2017.pdf...


Processing file: 51st_2017.pdf...
Processing file: 45th_2017.pdf...
Processing file: 27th_2017.pdf...
Processing file: 6th_2017.pdf...
Processing file: 50th_2017.pdf...
Processing file: 26th_2017.pdf...
Processing file: 40th_2017.pdf...
Processing file: 42th_2017.pdf...
Processing file: 7th_2017.pdf...
Processing file: 12th_2017.pdf...
Processing file: 1st_2017.pdf...
Processing file: 35th_2017.pdf...
Processing file: 37th_2017.pdf...
Processing file: 9th_2017.pdf...
Processing file: 24th _2017.pdf...
Processing file: 23rd _2017.pdf...
Processing file: 5th_2017.pdf...
Processing file: 11th_2017.pdf...
Processing file: 17th_2017.pdf...
Processing file: 16th_2017.pdf...
Processing file: 48th_2017.pdf...
Processing file: 29th_2017.pdf...
Processing file: 34th_2017.pdf...
Processing file: 44th_2017.pdf...
Processing file: 30th_2017.pdf...
Processing file: 36th_2017.pdf...
Processing file: 38th_2017.pdf...
Processing year: 2018...
Processing file: 45th_2018.pdf...
Processing file: 18th_2018

Processing file: 29th_2018.pdf...
Processing file: 34th_2018.pdf...
Processing file: 13th_2018.pdf...
Processing file: 10th_2018.pdf...
Processing file: 33rd_2018.pdf...
Processing file: 49th_2018.pdf...
Processing file: 16th_2018.pdf...
Processing file: 15th_2018.pdf...
Processing file: 11th_2018.pdf...
Processing file: 37th_2018.pdf...
Processing file: 26th_2018.pdf...
Processing file: 4th_2018.pdf...
Processing file: 39th_2018.pdf...
Processing file: 32nd_2018.pdf...
Processing file: 51th_2018.pdf...
Processing file: 21st_2018.pdf...
Processing file: 44th_2018.pdf...
Processing file: 46th_2018.pdf...
Processing file: 30th_2018.pdf...
Processing file: 2nd_2018.pdf...
Processing file: 23rd_2018.pdf...
Processing file: 47th_2018.pdf...
Processing file: 38th_2018.pdf...
Processing file: 8th_2018.pdf...
Processing file: 31st_2018.pdf...
Processing file: 1st_2018.pdf...
CPU times: user 7min 34s, sys: 338 ms, total: 7min 35s
Wall time: 7min 35s


After extracting the relevant information the list is transformed into a DataFrame and further inspected.

In [6]:
outbreaks_raw = pd.DataFrame(outbreaks_raw)

In [7]:
outbreaks_raw.shape

(2065, 10)

In [8]:
# make sure to display all dataframe columns
pd.set_option('display.max_columns', None)

In [9]:
outbreaks_raw.head()

,0,1,2,3,4,5,6,7,8,9
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf
1,positive,for,v.,cholerae,(ogawa,serotype),and,9,None,2010/31st_2010.pdf
2,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf
3,growth,of,v.,cholerae,(el,tor,-ogawa,serotype),None,2010/31st_2010.pdf
4,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf


In [10]:
# rename columns
column_names = []

for i in range(outbreaks_raw.shape[1]):
    column_names.append('col'+str(i))

outbreaks_raw.columns = column_names

In [11]:
outbreaks_raw.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf
1,positive,for,v.,cholerae,(ogawa,serotype),and,9,None,2010/31st_2010.pdf
2,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf
3,growth,of,v.,cholerae,(el,tor,-ogawa,serotype),None,2010/31st_2010.pdf
4,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf


In [12]:
# check which columns contain cholera 
for i in range(outbreaks_raw.shape[1]):
    print('Column {} contains cholera: {}'.format(i, 'cholera' in outbreaks_raw.iloc[:,i].unique()))

Column 0 contains cholera: True
Column 1 contains cholera: False
Column 2 contains cholera: False
Column 3 contains cholera: True
Column 4 contains cholera: True
Column 5 contains cholera: False
Column 6 contains cholera: False
Column 7 contains cholera: False
Column 8 contains cholera: False
Column 9 contains cholera: False


Check out the rows where cholera is mentioned in different columns to better understand the structure of the extracted information.

In [13]:
outbreaks_raw.loc[outbreaks_raw['col0'] == 'cholera']

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
268,cholera,culture,(v.,cholerae,ogawa).,health,camp,conducted,None,2011/36th_2011.pdf


In [14]:
outbreaks_raw.loc[outbreaks_raw['col3'] == 'cholera']

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf
2,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf
4,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf
5,growth,of,vibrio,cholera,01,on,culture,at,None,2010/39th_2010.pdf
7,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf
...,...,...,...,...,...,...,...,...,...,...
2058,positive,for,v.,cholera,sero,type,ogawa.,three,None,2018/8th_2018.pdf
2060,delhi,north,delhi,cholera,12,00,06-07-18,under,None,2018/31st_2018.pdf
2061,positive,for,v.,cholera,01,by,culture,in,None,2018/31st_2018.pdf
2062,mh/nas/2018/27/961,maharashtra,nashik,cholera,195,"03(55/m,","65/m,",30/f),None,2018/31st_2018.pdf


In [15]:
outbreaks_raw.loc[outbreaks_raw['col4'] == 'cholera']

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9
115,positive,for,vibrio,cholerae.,cholera,cases,occurred,due,None,2010/29th_2010.pdf


The column to focus on is col4. The rows where cholera is mentioned in other columns don't seem to be relevant as they are not the outbreak information we look for.

## Clean main cholera column

The main cholera column col4 contains in many cases much more than just a pure cholera string. Thus, all characters but cholera are removed.

In [16]:
outbreaks_raw['col3'].value_counts()

cholera                                                                          1259
cholera.                                                                          268
cholerae                                                                          102
cholerae.                                                                          77
cholera?                                                                           41
                                                                                 ... 
diarrheal–measlesdenguecholeramumpsanthraxjaundicehepatitis-ahepatitis-eacute       1
typhoidcholera?no.                                                                  1
diarrhealdiseasedenguecholerafevermalaria                                           1
ejaundicecholeramumps16129222111111no.                                              1
adiphtheriaacute–cholera15139653322111111no.                                        1
Name: col3, Length: 125, dtype: int64

In [17]:
outbreaks_raw['col3_clean'] = outbreaks_raw['col3'].apply(lambda x: 'cholera' if 'cholera' in x else x)

In [18]:
outbreaks_raw['col3_clean'].value_counts()

cholera    2065
Name: col3_clean, dtype: int64

## Filter outbreaks

We know that a cholera outbreak is mentioned when we see cholera in col4 and a number of cases and/or deaths right next to it. Therefore, we filter the rows that satisfy this condition.

In [19]:
# next to where a cholera outbreak is mentioned there should be a number indicating the cases and/or deaths
outbreaks_raw['outbreak'] = outbreaks_raw['col4'].apply(lambda x: 0 if re.search('[a-zA-Z]', str(x)) else 1)

In [20]:
# filter dataframe where an outbreaks appears to be mentioned
outbreaks = outbreaks_raw.loc[outbreaks_raw['outbreak'] == 1].reset_index(drop=True)

In [21]:
outbreaks.shape

(685, 12)

In [22]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1
3,growth,of,vibrio,cholera,01,on,culture,at,None,2010/39th_2010.pdf,cholera,1
4,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1


In [23]:
outbreaks.tail()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak
680,kl/mlp/2018/08/0241,kerala,malappuram,cholera,03,00,01-02-18,under,None,2018/8th_2018.pdf,cholera,1
681,delhi,north,delhi,cholera,12,00,06-07-18,under,None,2018/31st_2018.pdf,cholera,1
682,positive,for,v.,cholera,01,by,culture,in,None,2018/31st_2018.pdf,cholera,1
683,mh/nas/2018/27/961,maharashtra,nashik,cholera,195,"03(55/m,","65/m,",30/f),None,2018/31st_2018.pdf,cholera,1
684,positive,for,vibrio,cholerae,&,all,samples,negative,None,2018/31st_2018.pdf,cholera,1


The rows that we suspect to mention outbreaks appear to be fine in many cases, but not all as we will see later.

## Extract and clean start dates

Now we need to extract and clean the start dates of outbreaks. We know that the start date of an outbreak is supposed to be mentioned right next to the number of cases and/or deaths.

In [24]:
# clean columns suspected to contain start dates
for col in outbreaks.columns[5:8]:
    outbreaks[col+'_clean'] = outbreaks[col].apply(lambda x: None if re.search('[a-zA-Z]', str(x)) is not None
                                                   else (None if len(str(x)) < 4
                                                         else x.replace('(', '').replace(')', '').replace('*', '')))

In [25]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10
3,growth,of,vibrio,cholera,01,on,culture,at,None,2010/39th_2010.pdf,cholera,1,None,None,None
4,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10


In [26]:
# extract the first date mentioned from the suspected columns
start_dates = []

for i, row in outbreaks.iterrows():
    dates = row[12:]
    start_date = next((i for i in dates if i is not None), None)
    start_dates.append(start_date)

In [27]:
outbreaks['start_date'] = start_dates

In [28]:
# check extracted start dates
outbreaks['start_date'].unique()

array(['13.07.10', '20.07.10', '10.09.10', None, '06.09.10', '24.09.10',
       '03.07.10', '24.06.10', '29.06.10', '27.06.10', '19.12.10',
       '29.07.10', '15.06.10', '04.11.10', '10.05.10', '21.09.10',
       '28.12.09', '11.12.10', '24.01.10', '17.08.10', '16.07.10',
       '27.05.10', '30.05.10', '22.10.10', '03.10.10', '20.08.10',
       '18.08.10', '28.02.10', '04.06.10', '24.10.10', '22.06.10',
       '29.11.10', '16.08.10', '06.08.10', '12.05.10', '03.05.10',
       '23.04.10', '12.07.10', '06.07.10', '06.06.11', '08.07.11',
       '27.06.11', '26.06.11', '03.03.11', '13.07.11', '15.10.11',
       '24.10.11', '23.09.11', '01.10.11', '08.03.11', '08.12.11',
       '11.12.11', '23.04.11', '18.04.11', '24.04.11', '07.04.11',
       '20.06.11', '14.06.11', '20.02.11', '11.07.11', '23.07.11',
       '20.04.11', '07.10.11', '03.01.11', '12.12.11', '14.10.11',
       '09.09.11', '22.08.11', '18.08.11', '20.08.11', '16.08.11',
       '21.08.11', '14.08.11', '20.03.11', '15.09.11', '

We see that the dates appear in many different formats. Thus, we should harmonise them.

In [29]:
# drop all rows that do not contain any date
outbreaks = outbreaks.loc[outbreaks['start_date'].notnull()].reset_index(drop=True)

In [30]:
# harmonise date formats to extract year
outbreaks['year'] = outbreaks['start_date'].apply(lambda x: x.split('.')[2] if '.' in x
                                                  else (x.split('/')[2] if '/' in x
                                                        else (x.split('-')[2] if '-' in x
                                                              else x)))

In [31]:
# further harmonise date formats and extract years
outbreaks['year'] = outbreaks['year'].str.pad(width=3, side='left', fillchar='0')
outbreaks['year'] = outbreaks['year'].str.pad(width=4, side='left', fillchar='2')
outbreaks['year'] = outbreaks['year'].astype(np.int64)

In [32]:
# check years
outbreaks['year'].unique()

array([2010, 2009, 2011, 2012, 2013, 2014, 2015, 2019, 2016, 2017, 2018])

In [33]:
# check case where there seems to be a typo
outbreaks.loc[(outbreaks['col9'] == '2015/39th_2015.pdf') & (outbreaks['year'] == 2019)]

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year
393,karnataka,bagalkot,vii.,cholera,66,00,20-09-19,25-09-19,None,2015/39th_2015.pdf,cholera,1,None,20-09-19,25-09-19,20-09-19,2019


In [34]:
# fix typo
outbreaks.loc[(outbreaks['col9'] == '2015/39th_2015.pdf') & (outbreaks['year'] == 2019), 'year'] = 2015

In [35]:
# harmonise date formats to extract month
outbreaks['month'] = outbreaks['start_date'].apply(lambda x: x.split('.')[1] if '.' in x
                                                   else (x.split('/')[1] if '/' in x
                                                         else (x.split('-')[1] if '-' in x
                                                               else x)))

In [36]:
# cast month to integer
outbreaks['month'] = outbreaks['month'].astype(np.int64)

In [37]:
# check months
outbreaks['month'].unique()

array([ 7,  9,  6, 12, 11,  5,  1,  8, 10,  2,  4,  3])

In [38]:
outbreaks.shape

(587, 18)

In [39]:
# drop all rows that do not contain data from 2010 to 2018
outbreaks = outbreaks.loc[outbreaks['year'].isin(years)].reset_index(drop=True)

In [40]:
outbreaks.shape

(586, 18)

In [41]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None,13.07.10,2010,7
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None,20.07.10,2010,7
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10,10.09.10,2010,9
3,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10,06.09.10,2010,9
4,water.,bharuch,xii.,cholera,2,/,0,24.09.10,None,2010/41st_2010.pdf,cholera,1,None,None,24.09.10,24.09.10,2010,9


In [42]:
outbreaks.tail()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month
581,pb/ldh/2018/44/1456,punjab,ludhiana,cholera,12,00,12-09-18,under,None,2018/44th_2018.pdf,cholera,1,None,12-09-18,None,12-09-18,2018,9
582,west,bengal,bardhaman,cholera,69,00,02-06-18,04-06-18,None,2018/23rd_2018.pdf,cholera,1,None,02-06-18,04-06-18,02-06-18,2018,6
583,basis.,ch/chd/2018/22/760,chandigarh,cholera,01,00,23-05-18,under,None,2018/23rd_2018.pdf,cholera,1,None,23-05-18,None,23-05-18,2018,5
584,kl/mlp/2018/08/0241,kerala,malappuram,cholera,03,00,01-02-18,under,None,2018/8th_2018.pdf,cholera,1,None,01-02-18,None,01-02-18,2018,2
585,delhi,north,delhi,cholera,12,00,06-07-18,under,None,2018/31st_2018.pdf,cholera,1,None,06-07-18,None,06-07-18,2018,7


## Map states, districts and location (geometry)

Next, we need to map states and districts to their geographic location. We do this with the help of the [Database of Global Administrative Areas](https://www.gadm.org/index.html). We download the Level 2 administrative zones for India and explore them before we extract the relevant information and merge it with the outbreaks data.

In [43]:
!wget --recursive --no-directories --no-clobber --directory-prefix=../data/cholera_outbreaks https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_IND_shp.zip

File ‘../data/cholera_outbreaks/gadm36_IND_shp.zip’ already there; not retrieving.



In [44]:
!unzip -u -d ../data/cholera_outbreaks/gadm36_IND_shp ../data/cholera_outbreaks/gadm36_IND_shp.zip

Archive:  ../data/cholera_outbreaks/gadm36_IND_shp.zip


In [45]:
path = '../data/cholera_outbreaks/gadm36_IND_shp'

In [46]:
file = 'gadm36_IND_2.shp'

In [47]:
india = gpd.read_file(os.path.join(path, file))

In [48]:
# check the data's crs
india.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
india.shape

(666, 14)

In [50]:
india.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   GID_0      666 non-null    object  
 1   NAME_0     666 non-null    object  
 2   GID_1      666 non-null    object  
 3   NAME_1     666 non-null    object  
 4   NL_NAME_1  0 non-null      object  
 5   GID_2      666 non-null    object  
 6   NAME_2     666 non-null    object  
 7   VARNAME_2  190 non-null    object  
 8   NL_NAME_2  0 non-null      object  
 9   TYPE_2     666 non-null    object  
 10  ENGTYPE_2  666 non-null    object  
 11  CC_2       0 non-null      object  
 12  HASC_2     626 non-null    object  
 13  geometry   666 non-null    geometry
dtypes: geometry(1), object(13)
memory usage: 73.0+ KB


In [51]:
india.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.1_1,Nicobar Islands,None,None,District,District,None,IN.AN.NI,"MULTIPOLYGON (((93.78773 6.85264, 93.78849 6.8..."
1,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.2_1,North and Middle Andaman,None,None,District,District,None,IN.AN.NM,"MULTIPOLYGON (((92.93898 12.22386, 92.93916 12..."
2,IND,India,IND.1_1,Andaman and Nicobar,None,IND.1.3_1,South Andaman,None,None,District,District,None,IN.AN.SA,"MULTIPOLYGON (((92.47972 10.52056, 92.47945 10..."
3,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.1_1,Anantapur,"Anantpur, Ananthapur",None,District,District,None,IN.AD.AN,"POLYGON ((77.71420 13.76079, 77.71314 13.75074..."
4,IND,India,IND.2_1,Andhra Pradesh,None,IND.2.2_1,Chittoor,Chitoor|Chittor,None,District,District,None,IN.AD.CH,"POLYGON ((78.46293 12.63537, 78.46190 12.63228..."


In [52]:
# select states, districts and geometry
states_districts = india[['NAME_1', 'NAME_2', 'geometry']].copy()

In [53]:
# rename columns
states_districts.columns = ['state', 'district', 'geometry']

In [54]:
# make states and districts lowercase to simplify the mapping
states_districts['state'] = states_districts['state'].str.lower()
states_districts['district'] = states_districts['district'].str.lower()

In [55]:
# create list with unique districts
districts = states_districts['district'].unique().tolist()

In [56]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None,13.07.10,2010,7
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None,20.07.10,2010,7
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10,10.09.10,2010,9
3,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10,06.09.10,2010,9
4,water.,bharuch,xii.,cholera,2,/,0,24.09.10,None,2010/41st_2010.pdf,cholera,1,None,None,24.09.10,24.09.10,2010,9


In [57]:
# remove all but alphanumeric characters from columns suspected to contain states and districts
for col in outbreaks.columns[0:3]:
    outbreaks[col+'_clean'] = outbreaks[col].apply(lambda x: re.sub('\W', '', x))
    outbreaks[col+'_clean'] = outbreaks[col+'_clean'].apply(lambda x: None if len(x) < 3 else x)

In [58]:
# harmonise district names in col1_clean to simplify mapping
outbreaks.loc[outbreaks['col1_clean'] == 'mahabubnagar', 'col1_clean'] = 'mahbubnagar'
outbreaks.loc[outbreaks['col1_clean'] == 'ahmedabad', 'col1_clean'] = 'ahmadabad'
outbreaks.loc[outbreaks['col1_clean'] == 'howrah', 'col1_clean'] = 'haora'
outbreaks.loc[outbreaks['col1_clean'] == 'hooghly', 'col1_clean'] = 'hugli'
outbreaks.loc[outbreaks['col1_clean'] == 'hoogly', 'col1_clean'] = 'hugli'
outbreaks.loc[outbreaks['col1_clean'] == 'villupuram', 'col1_clean'] = 'viluppuram'
outbreaks.loc[outbreaks['col1_clean'] == 'haridwar', 'col1_clean'] = 'hardwar'
outbreaks.loc[outbreaks['col1_clean'] == 'davangere', 'col1_clean'] = 'davanagere'
outbreaks.loc[outbreaks['col1_clean'] == 'davengere', 'col1_clean'] = 'davanagere'
outbreaks.loc[outbreaks['col1_clean'] == 'davangare', 'col1_clean'] = 'davanagere'
outbreaks.loc[outbreaks['col1_clean'] == 'tiruchirapalli', 'col1_clean'] = 'tiruchirappalli'
outbreaks.loc[outbreaks['col1_clean'] == 'darang', 'col1_clean'] = 'darrang'
outbreaks.loc[outbreaks['col1_clean'] == 'virudhunager', 'col1_clean'] = 'virudunagar'
outbreaks.loc[outbreaks['col1_clean'] == 'chikkaballapur', 'col1_clean'] = 'chikballapura'
outbreaks.loc[outbreaks['col1_clean'] == 'gondia', 'col1_clean'] = 'gondiya'
outbreaks.loc[outbreaks['col1_clean'] == 'purulia', 'col1_clean'] = 'puruliya'
outbreaks.loc[outbreaks['col1_clean'] == 'kalaburagi', 'col1_clean'] = 'gulbarga'
outbreaks.loc[outbreaks['col1_clean'] == 'kalburgi', 'col1_clean'] = 'gulbarga'
outbreaks.loc[outbreaks['col1_clean'] == 'berhampur', 'col1_clean'] = 'ganjam'
outbreaks.loc[outbreaks['col1_clean'] == 'sholapur', 'col1_clean'] = 'solapur'
outbreaks.loc[outbreaks['col1_clean'] == 'raigad', 'col1_clean'] = 'raigarh'
outbreaks.loc[outbreaks['col1_clean'] == 'panchmahal', 'col1_clean'] = 'panch mahals'
outbreaks.loc[outbreaks['col1_clean'] == 'sibsagar', 'col1_clean'] = 'sivasagar'
outbreaks.loc[outbreaks['col1_clean'] == 'sibsagar', 'col1_clean'] = 'kabeerdham'
outbreaks.loc[outbreaks['col1_clean'] == 'banaskantha', 'col1_clean'] = 'banas kantha'
outbreaks.loc[outbreaks['col1_clean'] == 'chamarajnagar', 'col1_clean'] = 'chamrajnagar'
outbreaks.loc[outbreaks['col1_clean'] == 'khargaon', 'col1_clean'] = 'west nimar'
outbreaks.loc[outbreaks['col1_clean'] == 'mysuru', 'col1_clean'] = 'mysore'
outbreaks.loc[outbreaks['col1_clean'] == 'mohali', 'col1_clean'] = 'sahibzada ajit singh nagar'
outbreaks.loc[outbreaks['col1_clean'] == 'delhi', 'col1_clean'] = 'west'
outbreaks.loc[outbreaks['col1_clean'] == 'kawardha', 'col1_clean'] = 'kabeerdham'
outbreaks.loc[outbreaks['col1_clean'] == 'chirtadurga', 'col1_clean'] = 'chitradurga'
outbreaks.loc[outbreaks['col1_clean'] == 'budgam', 'col1_clean'] = 'badgam'
outbreaks.loc[outbreaks['col1_clean'] == 'gulburga', 'col1_clean'] = 'gulbarga'
outbreaks.loc[outbreaks['col1_clean'] == 'jangir', 'col1_clean'] = 'janjgir-champa'
outbreaks.loc[outbreaks['col1_clean'] == 'kancheepuramsaidapet', 'col1_clean'] = 'kancheepuram'

# harmonise district names in col2_clean to simplify mapping
outbreaks.loc[outbreaks['col2_clean'] == 'thiruvannamalai', 'col2_clean'] = 'tiruvannamalai'
outbreaks.loc[outbreaks['col2_clean'] == 'raigad', 'col2_clean'] = 'raigarh'
outbreaks.loc[outbreaks['col2_clean'] == 'davangere', 'col2_clean'] = 'davanagere'
outbreaks.loc[outbreaks['col2_clean'] == 'sibsagar', 'col2_clean'] = 'sivasagar'
outbreaks.loc[outbreaks['col2_clean'] == 'sabarkantha', 'col2_clean'] = 'sabar kantha'

In [59]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None,13.07.10,2010,7,himachal,pradesh,shimla
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None,20.07.10,2010,7,alwar,sikar,jaipur
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10,10.09.10,2010,9,assam,dibrugarh,xvi
3,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10,06.09.10,2010,9,karnataka,belgaum,xxi
4,water.,bharuch,xii.,cholera,2,/,0,24.09.10,None,2010/41st_2010.pdf,cholera,1,None,None,24.09.10,24.09.10,2010,9,water,bharuch,xii


In [60]:
outbreaks.columns[18:]

Index(['col0_clean', 'col1_clean', 'col2_clean'], dtype='object')

In [61]:
# extract district strings from columns suspected to contain districts
for col in outbreaks.columns[18:]:
    outbreaks[col+'_district'] = outbreaks[col].apply(lambda x: None if x is None else next((d for d in districts if x in str(d)), None))

In [62]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None,13.07.10,2010,7,himachal,pradesh,shimla,None,None,shimla
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None,20.07.10,2010,7,alwar,sikar,jaipur,alwar,sikar,jaipur
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10,10.09.10,2010,9,assam,dibrugarh,xvi,None,dibrugarh,None
3,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10,06.09.10,2010,9,karnataka,belgaum,xxi,None,belgaum,None
4,water.,bharuch,xii.,cholera,2,/,0,24.09.10,None,2010/41st_2010.pdf,cholera,1,None,None,24.09.10,24.09.10,2010,9,water,bharuch,xii,None,bharuch,None


In [63]:
# select processed columns suspected to contain districts
district_cols = [col for col in outbreaks.columns if 'district' in col]

In [64]:
# extract districts from processed columns suspected to contain districts
outbreaks_districts = []

for i, row in outbreaks.iterrows():
    all_district_cols = row[district_cols]
    district = next((i for i in all_district_cols if i is not None), None)
    outbreaks_districts.append(district)

In [65]:
# create new column with extracted districts
outbreaks['district'] = outbreaks_districts

In [66]:
# manually map missing districts
outbreaks.loc[(outbreaks['col4'] == '110') & (outbreaks['col6'] == '25/10/13'), 'district'] = 'bankura'
outbreaks.loc[(outbreaks['col4'] == '50') & (outbreaks['col6'] == '25/10/13') & (outbreaks['col7'] == '28/10/13'), 'district'] = 'bankura'
outbreaks.loc[(outbreaks['col4'] == '23') & (outbreaks['col6'] == '23/10/13') & (outbreaks['col7'] == '23/10/13'), 'district'] = 'puruliya'
outbreaks.loc[(outbreaks['col4'] == '13') & (outbreaks['col6'] == '31/07/13') & (outbreaks['col7'] == '31/07/13'), 'district'] = 'puruliya'
outbreaks.loc[(outbreaks['col4'] == '33') & (outbreaks['col6'] == '01/06/13') & (outbreaks['col7'] == '08/06/13'), 'district'] = 'chitradurga'
outbreaks.loc[(outbreaks['col4'] == '88') & (outbreaks['col6'] == '05/05/13'), 'district'] = 'sangli'
outbreaks.loc[(outbreaks['col4'] == '60') & (outbreaks['col6'] == '01/05/13') & (outbreaks['col7'] == '02/05/13'), 'district'] = 'puruliya'
outbreaks.loc[(outbreaks['col4'] == '36') & (outbreaks['col6'] == '19/04/13'), 'district'] = 'davanagere'
outbreaks.loc[(outbreaks['col4'] == '161') & (outbreaks['col6'] == '27/08/12'), 'district'] = 'nagpur'
outbreaks.loc[(outbreaks['col4'] == '73') & (outbreaks['col6'] == '22/07/12') & (outbreaks['col7'] == '23/07/12'), 'district'] = 'bankura'
outbreaks.loc[(outbreaks['col4'] == '19') & (outbreaks['col6'] == '21/07/12') & (outbreaks['col7'] == '21/07/12'), 'district'] = 'chikmagalur'
outbreaks.loc[(outbreaks['col4'] == '21') & (outbreaks['col6'] == '06/07/12') & (outbreaks['col7'] == '12/07/12'), 'district'] = 'birbhum'
outbreaks.loc[(outbreaks['col4'] == '59') & (outbreaks['col6'] == '25/05/12') & (outbreaks['col7'] == '25/05/12'), 'district'] = 'tumkur'
outbreaks.loc[(outbreaks['col4'] == '8') & (outbreaks['col6'] == '03/05/12') & (outbreaks['col7'] == '07/05/12'), 'district'] = 'wayanad'
outbreaks.loc[(outbreaks['col4'] == '26') & (outbreaks['col7'] == '16.08.11'), 'district'] = 'birbhum'
outbreaks.loc[(outbreaks['col4'] == '86') & (outbreaks['col6'] == '25/10/13') & (outbreaks['col7'] == '28/10/13'), 'district'] = 'bankura'
outbreaks.loc[(outbreaks['col4'] == '25') & (outbreaks['col6'] == '19/05/12'), 'district'] = 'mandya'
outbreaks.loc[(outbreaks['col4'] == '12') & (outbreaks['col6'] == '13.03.12') & (outbreaks['col7'] == '14.03.12'), 'district'] = 'mysore'

In [67]:
outbreaks.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
0,himachal,pradesh,shimla,cholera,235/,1*,13.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,13.07.10,None,13.07.10,2010,7,himachal,pradesh,shimla,None,None,shimla,shimla
1,"alwar,","sikar,",jaipur,cholera,1095/0,#,20.07.10,under,None,2010/31st_2010.pdf,cholera,1,None,20.07.10,None,20.07.10,2010,7,alwar,sikar,jaipur,alwar,sikar,jaipur,alwar
2,assam,dibrugarh,xvi.,cholera,10,/,0,10.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,10.09.10,10.09.10,2010,9,assam,dibrugarh,xvi,None,dibrugarh,None,dibrugarh
3,karnataka,belgaum,xxi.,cholera,19,/,0,06.09.10,None,2010/39th_2010.pdf,cholera,1,None,None,06.09.10,06.09.10,2010,9,karnataka,belgaum,xxi,None,belgaum,None,belgaum
4,water.,bharuch,xii.,cholera,2,/,0,24.09.10,None,2010/41st_2010.pdf,cholera,1,None,None,24.09.10,24.09.10,2010,9,water,bharuch,xii,None,bharuch,None,bharuch


In [68]:
outbreaks.tail()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
581,pb/ldh/2018/44/1456,punjab,ludhiana,cholera,12,00,12-09-18,under,None,2018/44th_2018.pdf,cholera,1,None,12-09-18,None,12-09-18,2018,9,pbldh2018441456,punjab,ludhiana,None,None,ludhiana,ludhiana
582,west,bengal,bardhaman,cholera,69,00,02-06-18,04-06-18,None,2018/23rd_2018.pdf,cholera,1,None,02-06-18,04-06-18,02-06-18,2018,6,west,bengal,bardhaman,west godavari,None,None,west godavari
583,basis.,ch/chd/2018/22/760,chandigarh,cholera,01,00,23-05-18,under,None,2018/23rd_2018.pdf,cholera,1,None,23-05-18,None,23-05-18,2018,5,basis,chchd201822760,chandigarh,None,None,chandigarh,chandigarh
584,kl/mlp/2018/08/0241,kerala,malappuram,cholera,03,00,01-02-18,under,None,2018/8th_2018.pdf,cholera,1,None,01-02-18,None,01-02-18,2018,2,klmlp2018080241,kerala,malappuram,None,None,malappuram,malappuram
585,delhi,north,delhi,cholera,12,00,06-07-18,under,None,2018/31st_2018.pdf,cholera,1,None,06-07-18,None,06-07-18,2018,7,delhi,north,delhi,None,north and middle andaman,None,north and middle andaman


In [69]:
outbreaks.shape

(586, 25)

In [70]:
# keep only rows that have a district
outbreaks = outbreaks.loc[outbreaks['district'].notnull()]

In [71]:
outbreaks.shape

(564, 25)

In [72]:
# map states and geometry to districts
outbreaks_mapped = pd.merge(states_districts, outbreaks, how='right', on='district')[['state', 'district', 'year', 'month', 'outbreak', 'geometry']].reset_index(drop=True)

In [73]:
outbreaks_mapped.shape

(573, 6)

In [74]:
outbreaks_mapped

,state,district,year,month,outbreak,geometry
0,himachal pradesh,shimla,2010,7,1,"POLYGON ((77.51051 30.78743, 77.50089 30.79253..."
1,rajasthan,alwar,2010,7,1,"POLYGON ((76.45745 27.08288, 76.44963 27.08627..."
2,assam,dibrugarh,2010,9,1,"POLYGON ((95.37698 27.10218, 95.37267 27.10167..."
3,karnataka,belgaum,2010,9,1,"POLYGON ((74.75089 15.49199, 74.74472 15.48939..."
4,gujarat,bharuch,2010,9,1,"MULTIPOLYGON (((72.67597 21.44681, 72.67597 21..."
...,...,...,...,...,...,...
568,punjab,ludhiana,2018,9,1,"POLYGON ((76.07124 30.56471, 76.06435 30.56714..."
569,andhra pradesh,west godavari,2018,6,1,"POLYGON ((81.81902 16.66214, 81.83632 16.64615..."
570,chandigarh,chandigarh,2018,5,1,"POLYGON ((76.80293 30.67548, 76.79437 30.66932..."
571,kerala,malappuram,2018,2,1,"POLYGON ((76.06149 10.72085, 76.05975 10.71879..."


In [75]:
# find districts with identical names appearing in different states
duplicate_districts = states_districts['district'].value_counts()[states_districts['district'].value_counts() > 1].index.tolist()
outbreaks_mapped[outbreaks_mapped['district'].isin(duplicate_districts)].sort_values('district')

,state,district,year,month,outbreak,geometry
517,maharashtra,aurangabad,2017,11,1,"POLYGON ((75.21554 19.55770, 75.20956 19.56258..."
446,maharashtra,aurangabad,2016,7,1,"POLYGON ((75.21554 19.55770, 75.20956 19.56258..."
516,bihar,aurangabad,2017,11,1,"POLYGON ((84.73145 25.05792, 84.73087 25.05387..."
445,bihar,aurangabad,2016,7,1,"POLYGON ((84.73145 25.05792, 84.73087 25.05387..."
163,chhattisgarh,bijapur,2012,8,1,"POLYGON ((81.12064 19.25221, 81.12340 19.24678..."
164,karnataka,bijapur,2012,8,1,"POLYGON ((76.41283 16.58693, 76.42019 16.57418..."
199,chhattisgarh,bijapur,2012,7,1,"POLYGON ((81.12064 19.25221, 81.12340 19.24678..."
200,karnataka,bijapur,2012,7,1,"POLYGON ((76.41283 16.58693, 76.42019 16.57418..."
443,chhattisgarh,bijapur,2016,3,1,"POLYGON ((81.12064 19.25221, 81.12340 19.24678..."
444,karnataka,bijapur,2016,3,1,"POLYGON ((76.41283 16.58693, 76.42019 16.57418..."


We need to check to which state the duplicated districts actually belong.

In [76]:
outbreaks.loc[(outbreaks['district'] == 'aurangabad') & (outbreaks['month'] == 11)] # maharashtra

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
530,mh/agb/2017/46/1591,maharashtra,aurangabad,cholera,5825,00,10-11-17,13-11-17,None,2017/46th_2017.pdf,cholera,1,None,10-11-17,13-11-17,10-11-17,2017,11,mhagb2017461591,maharashtra,aurangabad,None,None,aurangabad,aurangabad


In [77]:
outbreaks.loc[(outbreaks['district'] == 'aurangabad') & (outbreaks['month'] == 7)] # maharashtra

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
448,mh/agb/2016/27/1592,maharashtra,aurangabad,cholera,09,01,08-07-16,09-07-16,None,2016/27th_2016.pdf,cholera,1,None,08-07-16,09-07-16,08-07-16,2016,7,mhagb2016271592,maharashtra,aurangabad,None,None,aurangabad,aurangabad


In [78]:
outbreaks.loc[(outbreaks['district'] == 'bijapur') & (outbreaks['month'] == 8)] # karnataka

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
162,given.,bijapur,xvi,cholera,85,0,20/08/12,20/08/12,None,2012/34th_2012.pdf,cholera,1,None,20/08/12,20/08/12,20/08/12,2012,8,given,bijapur,xvi,None,bijapur,None,bijapur


In [79]:
outbreaks.loc[(outbreaks['district'] == 'bijapur') & (outbreaks['month'] == 7)] # karnataka

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
197,imparted.,bijapur,xi,cholera,32,0,08/07/12,12/07/12,None,2012/28th_2012.pdf,cholera,1,None,08/07/12,12/07/12,08/07/12,2012,7,imparted,bijapur,None,None,bijapur,None,bijapur


In [80]:
outbreaks.loc[(outbreaks['district'] == 'bijapur') & (outbreaks['month'] == 8)] # karnataka

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
162,given.,bijapur,xvi,cholera,85,0,20/08/12,20/08/12,None,2012/34th_2012.pdf,cholera,1,None,20/08/12,20/08/12,20/08/12,2012,8,given,bijapur,xvi,None,bijapur,None,bijapur


In [81]:
outbreaks.loc[(outbreaks['district'] == 'pratapgarh') & (outbreaks['month'] == 7)] # rajasthan

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
487,rj/pgr/2016/30/1789,rajasthan,pratapgarh,cholera,49,00,29-07-16,29-07-16,None,2016/30th_2016.pdf,cholera,1,None,29-07-16,29-07-16,29-07-16,2016,7,rjpgr2016301789,rajasthan,pratapgarh,None,None,pratapgarh,pratapgarh


In [82]:
outbreaks.loc[(outbreaks['district'] == 'raigarh') & (outbreaks['month'] == 7)] # maharashtra

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
72,maharashtra,raigad,xxii,cholera,67,/,0,11.07.11,None,2011/29th_2011.pdf,cholera,1,None,None,11.07.11,11.07.11,2011,7,maharashtra,raigarh,xxii,None,raigarh,None,raigarh
98,week.,11,raigad,cholera,94,/,0,11.07.11,None,2011/30th_2011.pdf,cholera,1,None,None,11.07.11,11.07.11,2011,7,week,None,raigarh,None,None,raigarh,raigarh


In [83]:
outbreaks.loc[(outbreaks['district'] == 'raigarh') & (outbreaks['month'] == 3)] # maharashtra

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col3_clean,outbreak,col5_clean,col6_clean,col7_clean,start_date,year,month,col0_clean,col1_clean,col2_clean,col0_clean_district,col1_clean_district,col2_clean_district,district
386,maharashtra,raigad,xxxi.,cholera,22,00,30/03/15,under,None,2015/16th_2015.pdf,cholera,1,None,30/03/15,None,30/03/15,2015,3,maharashtra,raigarh,xxxi,None,raigarh,None,raigarh


We figured out to which state the duplicated districts actually belong and drop the rows that contain wrong data.

In [84]:
outbreaks_mapped = outbreaks_mapped.drop(outbreaks_mapped.loc[(outbreaks_mapped['state'].isin(['bihar', 'chhattisgarh', 'uttar pradesh'])) & (outbreaks_mapped['district'].isin(['aurangabad', 'bijapur', 'raigarh', 'pratapgarh']))].index)

In [85]:
outbreaks_mapped.shape

(564, 6)

In [86]:
outbreaks_mapped.drop_duplicates().shape

(466, 6)

Finally, we drop duplicated rows since we want to count an outbreak only once per district and month and save the data in shapefile format.

In [87]:
outbreaks_mapped.drop_duplicates().reset_index(drop=True).to_file('../data/cholera_outbreaks/monthly_cholera_outbreaks_per_district_2010_2015.shp')